In [1]:
import h5py as h5
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import sys
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
from auton_survival.preprocessing import Scaler
import optuna
from sklearn.model_selection import ParameterGrid
#sys.path.append('/projects/EKOLEMEN/survival_tm/train_models/auton-survival')
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
from sklearn.model_selection import ParameterGrid
from auton_survival.estimators import SurvivalModel
from auton_survival.metrics import survival_regression_metric
from auton_survival.models.dsm import DeepSurvivalMachines
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

/home/hf8585/.conda/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
shots = np.load('/projects/EKOLEMEN/survival_tm/shots.npy')
tm_shots = np.load('/projects/EKOLEMEN/survival_tm/tm_shots.npy')
st_shots = np.load('/projects/EKOLEMEN/survival_tm/st_shots.npy')

def load_data(data_type):
    with open(f'/projects/EKOLEMEN/survival_tm/formatted_labels/{data_type}.pkl', 'rb') as f:
        data = pickle.load(f)
    
    return data['x'], data['t'], data['e']

# Don't want whole dataset right now so 0.5 factor to reduce
n = len(shots)

tr_size = int(n*0.80)
vl_size = int(n*0.10)
te_size = int(n*0.10)

train_shots = shots[:tr_size]
test_shots = shots[-te_size:]
valid_shots = shots[tr_size:tr_size+vl_size]

x_train, t_train, e_train = load_data('train')
x_test,  t_test,  e_test  = load_data('test')
x_valid, t_valid, e_valid = load_data('valid')

# Get inds for time <600ms
# is this needed? Not sure.
'''inds = np.where(t_train < 600)[0]

x_train = x_train[inds]
t_train = t_train[inds]
e_train = e_train[inds]'''

tm_inds = np.where(e_train == 1)[0]
st_inds = np.where(e_train == 0)[0]
new_st_inds = np.random.choice(st_inds, size=len(tm_inds), replace=False)
combined_inds = np.concatenate((tm_inds, new_st_inds))
sorted_combined_inds = np.sort(combined_inds)

x_train = x_train[sorted_combined_inds]
t_train = t_train[sorted_combined_inds]
e_train = e_train[sorted_combined_inds]

'''x_train = np.concatenate((x_train[tm_inds], x_train[new_st_inds]), axis=0)
t_train = np.concatenate((t_train[tm_inds], t_train[new_st_inds]), axis=0)
e_train = np.concatenate((e_train[tm_inds], e_train[new_st_inds]), axis=0)
plt.plot(t_train[10000:11000])
# Shuffle arrays because currently all 1s followed by all 0s
p = np.random.permutation(len(t_train))
x_train = x_train[p,:]
t_train = t_train[p]
e_train = e_train[p]'''

x_train_df = pd.DataFrame(x_train)
t_train_df = pd.DataFrame(t_train)
e_train_df = pd.DataFrame(e_train)

x_valid_df = pd.DataFrame(x_valid)
t_valid_df = pd.DataFrame(t_valid)
e_valid_df = pd.DataFrame(e_valid)

x_test_df = pd.DataFrame(x_test)
t_test_df = pd.DataFrame(t_test)
e_test_df = pd.DataFrame(e_test)

outcomes_valid_df = pd.DataFrame({'time': t_valid, 'event': e_valid})

outcomes_train_df = pd.DataFrame({'time': t_train, 'event': e_train})
outcomes_test_df = pd.DataFrame({'time': t_test, 'event': e_test})
#normalize
scaler = Scaler()
transformer = scaler.fit(x_train_df)
x_train_df_normed = transformer.transform(x_train_df)
x_valid_df_normed = transformer.transform(x_valid_df)
x_test_df_normed = transformer.transform(x_test_df)

'''with open('data/x_train_df_normed.pkl', 'wb') as f:
    pickle.dump(x_train_df_normed, f)
with open('data/x_valid_df_normed.pkl', 'wb') as f:
    pickle.dump(x_valid_df_normed, f)
with open('data/x_test_df_normed.pkl', 'wb') as f:
    pickle.dump(x_test_df_normed, f)
with open('data/outcomes_valid_df.pkl', 'wb') as f:
    pickle.dump(outcomes_valid_df, f)
with open('data/outcomes_train_df.pkl', 'wb') as f:
    pickle.dump(outcomes_train_df, f)
with open('data/outcomes_test_df.pkl', 'wb') as f:
    pickle.dump(outcomes_test_df, f)'''

"with open('data/x_train_df_normed.pkl', 'wb') as f:\n    pickle.dump(x_train_df_normed, f)\nwith open('data/x_valid_df_normed.pkl', 'wb') as f:\n    pickle.dump(x_valid_df_normed, f)\nwith open('data/x_test_df_normed.pkl', 'wb') as f:\n    pickle.dump(x_test_df_normed, f)\nwith open('data/outcomes_valid_df.pkl', 'wb') as f:\n    pickle.dump(outcomes_valid_df, f)\nwith open('data/outcomes_train_df.pkl', 'wb') as f:\n    pickle.dump(outcomes_train_df, f)\nwith open('data/outcomes_test_df.pkl', 'wb') as f:\n    pickle.dump(outcomes_test_df, f)"

In [27]:
# we give each timestep all the info from a few timesteps in the past. 

x_train_np = np.array(x_train)
t_train_np = np.array(t_train)[4:]
e_train_np = np.array(e_train)[4:]

x_valid_np = np.array(x_valid)
t_valid_np = np.array(t_valid)[4:]
e_valid_np = np.array(e_valid)[4:]

x_test_np = np.array(x_test)
t_test_np = np.array(t_test)[4:]
e_test_np = np.array(e_test)[4:]

outcomes_valid_memory = pd.DataFrame({'time': t_valid_np, 'event': e_valid_np})
outcomes_test_memory = pd.DataFrame({'time': t_test_np, 'event': e_test_np})
outcomes_train_memory = pd.DataFrame({'time': t_train_np, 'event': e_train_np})

x_valid_memory = np.array([x_valid_np[i-4:i+1].flatten() for i in range(4, len(x_valid_np))])
x_test_memory = np.array([x_test_np[i-4:i+1].flatten() for i in range(4, len(x_test_np))])
x_train_memory = np.array([x_train_np[i-4:i+1].flatten() for i in range(4, len(x_train_np))])

train_peaks = np.array(find_peaks_in_data(np.array(outcomes_train_memory['time'])))
valid_peaks = np.array(find_peaks_in_data(np.array(outcomes_valid_memory['time'])))
test_peaks = np.array(find_peaks_in_data(np.array(outcomes_test_memory['time'])))

new_train_peaks = np.sort(np.concatenate((train_peaks, train_peaks+1, train_peaks+2, train_peaks+3)))
new_valid_peaks = np.sort(np.concatenate((valid_peaks, valid_peaks+1, valid_peaks+2, valid_peaks+3)))
new_test_peaks = np.sort(np.concatenate((test_peaks, test_peaks+1, test_peaks+2, test_peaks+3)))


x_valid_memory = pd.DataFrame(np.delete(x_valid_memory, new_valid_peaks, axis=0))
x_test_memory = pd.DataFrame(np.delete(x_test_memory, new_test_peaks, axis=0))
x_train_memory = pd.DataFrame(np.delete(x_train_memory, new_train_peaks, axis=0))

scaler = Scaler()
transformer = scaler.fit(x_train_memory)
x_train_memory_normed = transformer.transform(x_train_memory)
x_valid_memory_normed = transformer.transform(x_valid_memory)
x_test_memory_normed = transformer.transform(x_test_memory)

with open('data/x_valid_memory_normed.pkl', 'wb') as f:
    pickle.dump(x_valid_memory_normed, f)
with open('data/x_test_memory_normed.pkl', 'wb') as f:
    pickle.dump(x_test_memory_normed, f)
with open('data/x_train_memory_normed.pkl', 'wb') as f:
    pickle.dump(x_train_memory_normed, f)
with open('data/outcomes_valid_memory.pkl', 'wb') as f:
    pickle.dump(outcomes_valid_memory, f)
with open('data/outcomes_train_memory.pkl', 'wb') as f:
    pickle.dump(outcomes_train_memory, f)
with open('data/outcomes_test_memory.pkl', 'wb') as f:
    pickle.dump(outcomes_test_memory, f)

In [36]:
print(train_peaks[0])
print(t_train_np[16])


17
120


In [37]:
signals = ['pinj', 'tinj', 'betan_EFIT01', 'qmin_EFIT01', 'ech_pwr_total', 'ip', 'bt', 'li_EFIT01', 'aminor_EFIT01', 
          'rmaxis_EFIT01', 'tribot_EFIT01', 'tritop_EFIT01', 'kappa_EFIT01', 'volume_EFIT01']

# we give each timestep all the info from a few timesteps in the past. 

x_train_np = np.array(x_train)
t_train_np = np.array(t_train)[:-4]
e_train_np = np.array(e_train)[:-4]

x_valid_np = np.array(x_valid)
t_valid_np = np.array(t_valid)[:-4]
e_valid_np = np.array(e_valid)[:-4]

x_test_np = np.array(x_test)
t_test_np = np.array(t_test)[:-4]
e_test_np = np.array(e_test)[:-4]

outcomes_valid_future = pd.DataFrame({'time': t_valid_np, 'event': e_valid_np})
outcomes_test_future = pd.DataFrame({'time': t_test_np, 'event': e_test_np})
outcomes_train_future = pd.DataFrame({'time': t_train_np, 'event': e_train_np})

# we keep the present and 4 in the future
x_valid_future = pd.DataFrame([np.concatenate((x_valid_np[i:i+5, 0:len(signals)].flatten(), x_valid_np[i, len(signals):])) for i in range(len(x_valid_np)-4)])
x_test_future = pd.DataFrame([np.concatenate((x_test_np[i:i+5, 0:len(signals)].flatten(), x_test_np[i, len(signals):])) for i in range(len(x_test_np)-4)])
x_train_future = pd.DataFrame([np.concatenate((x_train_np[i:i+5, 0:len(signals)].flatten(), x_train_np[i, len(signals):])) for i in range(len(x_train_np)-4)])

train_peaks = np.array(find_peaks_in_data(np.array(outcomes_train_future['time'])))
valid_peaks = np.array(find_peaks_in_data(np.array(outcomes_valid_future['time'])))
test_peaks = np.array(find_peaks_in_data(np.array(outcomes_test_future['time'])))

new_train_peaks = np.sort(np.concatenate((train_peaks-1, train_peaks-2, train_peaks-3, train_peaks-4)))
new_valid_peaks = np.sort(np.concatenate((valid_peaks-1, valid_peaks-2, valid_peaks-3, valid_peaks-4)))
new_test_peaks = np.sort(np.concatenate((test_peaks-1, test_peaks-2, test_peaks-3, test_peaks-4)))

x_valid_future = pd.DataFrame(np.delete(x_valid_future, new_valid_peaks, axis=0))
x_test_future = pd.DataFrame(np.delete(x_test_future, new_test_peaks, axis=0))
x_train_future = pd.DataFrame(np.delete(x_train_future, new_train_peaks, axis=0))

scaler = Scaler()
transformer = scaler.fit(x_train_future)
x_train_future_normed = transformer.transform(x_train_future)
x_valid_future_normed = transformer.transform(x_valid_future)
x_test_future_normed = transformer.transform(x_test_future)

with open('data/x_valid_future_normed.pkl', 'wb') as f:
    pickle.dump(x_valid_future_normed, f)
with open('data/x_test_future_normed.pkl', 'wb') as f:
    pickle.dump(x_test_future_normed, f)
with open('data/x_train_future_normed.pkl', 'wb') as f:
    pickle.dump(x_train_future_normed, f)
with open('data/outcomes_valid_future.pkl', 'wb') as f:
    pickle.dump(outcomes_valid_future, f)
with open('data/outcomes_train_future.pkl', 'wb') as f:
    pickle.dump(outcomes_train_future, f)
with open('data/outcomes_test_future.pkl', 'wb') as f:
    pickle.dump(outcomes_test_future, f)

In [6]:
# make WTC curve

# one issue with this metric is if we never predict TMs then we're always perfect. 
def fpr_auc(model, normed_x, normed_t, normed_e, prediction_times, threshold=0.7):
    out_survival = model.predict_survival(normed_x, prediction_times)
    fprs = []
    for i, time in enumerate(prediction_times):
        survival_prediction = out_survival[:,i]
        survival_prediction = (survival_prediction < threshold).astype(int)
        false_positives = np.logical_and(survival_prediction == 1, normed_e == 0)
        true_negatives = np.logical_and(survival_prediction == 0, normed_e == 0)
        fpr = false_positives.sum() / (false_positives.sum() + true_negatives.sum())
        fprs.append(fpr)

    auc = np.trapz(fprs, prediction_times)
    return auc, fprs, prediction_times

def find_peaks_in_data(data):
    peaks = []
    for i in range(1, len(data) - 1):
        if data[i-1] < data[i] > data[i+1]:
            peaks.append(i)
    return peaks

def fnr_auc(model, normed_x, normed_t, normed_e, prediction_times, threshold=0.7):
    out_survival = model.predict_survival(normed_x, prediction_times)
    fnrs = []
    fprs = []
    shot_indices = find_peaks_in_data(normed_t)
    for i, time in enumerate(prediction_times):
        tm_prediction_per_shot = []
        # 1 means correct TM prediction, 0 means unpredicted TM, -1 means no TM in shot and no TM predicted, -2 means no TM in shot and yes TM predicted
        # a TM is predicted when the survival prediction is 0 at any point in the shot. Check if better results when TM is consecutive 0s
        for j, shot_index in enumerate(shot_indices[:-1]):
            survival_prediction = out_survival[:,i]
            survival_prediction = (survival_prediction < threshold).astype(int)
            tm = (0 in survival_prediction[shot_indices[j]:shot_indices[j+1]])
            if normed_e[shot_index] == 1 and tm:
                tm_prediction_per_shot.append(1)
            elif normed_e[shot_index] == 1 and not tm:
                tm_prediction_per_shot.append(0)
            elif normed_e[shot_index] == 0 and not tm:
                tm_prediction_per_shot.append(-1)
            elif normed_e[shot_index] == 0 and tm:
                tm_prediction_per_shot.append(-2)
        fnr = tm_prediction_per_shot.count(0) / (tm_prediction_per_shot.count(1) + tm_prediction_per_shot.count(0))
        fpr = tm_prediction_per_shot.count(-2) / (tm_prediction_per_shot.count(-1) + tm_prediction_per_shot.count(-2))
        fnrs.append(fnr)
        fprs.append(fpr)
    auc_fnr = np.trapz(fnrs, prediction_times)
    auc_fpr = np.trapz(fprs, prediction_times)
    return auc_fpr, auc_fnr, fprs, fnrs, prediction_times

In [35]:
prediction_times = [20, 50, 100, 200]

In [ ]:
with open('survival.pkl', 'rb') as f:
    out_survival = pickle.load(f)
with open('models.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
model = loaded_model[8][0]
out_survival = model.predict_survival(x_test_df, prediction_times)
peaks = find_peaks_in_data(t_test)
for i in range(0, 3):
    peak_number = 140 + i
    start_index = peaks[peak_number]
    end_index = peaks[peak_number + 1]
    times = np.arange(0, (end_index - start_index)*20, 20)
    plt.plot(times, out_survival[start_index:end_index,2], label='Survival in 100ms')
    plt.plot(times, out_survival[start_index:end_index,3], label='Survival in 200ms')

    if (e_test[start_index]==1):
        plt.title('YES TM')
    else:
        plt.title('NO TM')
    plt.xlabel('Time / ms')
    plt.ylabel('Survival probability')
    plt.ylim(0.5, 1)
    plt.legend()
    plt.show()

In [36]:
threshold = 0.9
#fpr_auc, fprs, prediction_times = fpr_auc(model, x_test_df, t_test, e_test, prediction_times, threshold=threshold)
fpr_auc, fnr_auc, fprs, fnrs, predicted_times = fnr_auc(model, x_test_df, t_test, e_test, prediction_times, threshold=threshold)

/home/hf8585/TMPredictor/survival_tm/auton-survival/auton_survival/estimators.py:206: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return survival_predictions.sort_index(axis=0).interpolate().interpolate(method='bfill').T[times].values


In [37]:
print(fpr_auc, fnr_auc)
print(fprs)
print(fnrs)

179.9627143922446 1.358695652173913
[1.0, 1.0, 1.0, 0.9992542878448919]
[0.0, 0.0, 0.0015527950310559005, 0.024844720496894408]
